# 第6章：概率图模型

## 6.1 概述

**概率图模型**（probabilistic graphical models）：在概率模型的基础上，使用基于图的方法来表示概率分布（或者概率密度、密度函数），是一种通用化的`不确定性知识`表示和处理方法。

在概率图模型的表达中，结点表示变量，结点之间直接相连的边表示相应变量之间的概率关系。当概率分布$P$被表示成概率图模型之后，可以回答与概率分布$P$有关的问题，如计算条件概率$P(Y| E = e)$：在证据$e$给定的条件下，$Y$出现的边缘概率；推断使$P(X_{1}, X_{2}, \cdots, X_{n} | e)$最大的$(X_{1}, X_{2}, \cdots, X_{n})$的分布，即推断最大后验概率时的分布$\argmax_{X}P(X | e)$。

例如，假设$S$为一个汉语句子，$X$是句子$S$切分出来的词序列，则汉语句子的分词过程可以看成是推断使$P(X | S)$最大的词序列$X$的分布。在词性标注中，可以看作在给定序列$X$的情况下，寻找一组最可能的词性标签分布$T$，使得后验概率$P(T | X)$最大。

根据图模型（graphicalmodels）的边是否有向，概率图模型通常被划分成有向概率图模型和无向概率图模型。

<img src="./img/fig_6_1.jpg" width="500" />

**动态贝叶斯网络**（dynamic Bayesian networks，DBN）用于处理随时间变化的动态系统中的推断和预测问题。其中，**隐马尔可夫模型**（hidden Markov model，HMM）在语音识别、汉语自动分词与词性标注和统计机器翻译等若干语音语言处理任务中得到了应用；卡尔曼滤波器在信号处理领域得到了应用。

**马尔可夫网络**（Markov network）又称**马尔可夫随机场**（Markovrandomfield，MRF）。其中，**条件随机场**（conditional random field，CRF）应用于自然语言处理中的序列标注、特征选择、机器翻译等任务；**波尔兹曼机**（Boltzmannmachine）被用于依存句法分析和语义角色标注等。

概率图模型的演变：

* 横向：由点到线（序列结构）、线到面（图结构）。以朴素贝叶斯模型为基础的隐马尔可夫模型用于处理线性序列问题，有向图模型用于解决一般图问题；以逻辑回归模型为基础的线性链式条件随机场用于解决“线式”序列问题，通用条件随机场用于解决一般图问题。

* 纵向：在一定条件下生成式模型（generative mode）转变为判别式模型（discriminative model），朴素贝叶斯模型演变为逻辑回归模型，隐马尔可夫模型演变为以逻辑回归为基础的线性链式条件随机场，生成式有向图模型演变为通用条件随机场。

<img src="./img/fig_6_2.jpg" width="550" />

**生成式模型**（产生式模型）与**判别式模型**（区分式模型）的本质区别在于模型中观测序列$x$和状态序列$y$之间的决定关系，前者假设$y$决定$x$，后者假设$x$决定$y$。生成模型以“状态（输出）序列$y$按照一定的规律生成观测（输入）序列$x$”为假设，针对联合分布$p(x, y)$进行建模，并且通过估计使生成概率最大的生成序列来获取y。生成式模型是所有变量的全概率模型，因此可以模拟（生成）所有变量的值。在这类模型中一般都有严格的独立性假设，特征是事先给定的，并且特征之间的关系直接体现在公式中。这类模型的优点是：处理单类问题时比较灵活，模型变量之间的关系比较清楚，模型可以通过增量学习获得，可用于数据不完整的情况。其弱点在于模型的推导和学习比较复杂。

典型生成式模型有：$n$元语法模型、HMM朴素的贝叶斯分类器、概率上下文无关文法等。

**判别式模型**符合传统的模式分类思想，认为$y$由$x$决定，直接对后验概率$p(y | x)$进行建模，从$x$中提取特征、学习模型参数，使得条件概率符合一定形式的最优。这类模型的特征可以任意给定，一般特征是通过函数表示的。这种模型的优点是：处理多类问题或分辨某一类与其他类之间的差异时比较灵活，模型简单，容易建立和学习。其弱点为模型的描述能力有限，变量之间的关系不清楚，而且大多数区分式模型是有监督的学习方法，不能扩展成无监督的学习方法。

典型判别式模型有：最大熵模型、条件随机场、支持向量机、最大熵马尔可夫模型（maximum-entropy Markov model，MEMM）、感知机（perceptron）等。


## 6.2 贝叶斯网络

**贝叶斯网络**[Judea Pearl, 1985]又称为信度网络或信念网络（belief networks），是一种基于概率推理的数学模型，通过概率推理处理不确定性和不完整性问题，其理论基础是贝叶斯公式。

形式上，贝叶斯网络是一个有向无环图（directed acyelic graph，DAG），结点表示随机变量，可以是可观测量、隐含变量、未知参量或假设等；结点之间的有向边表示条件依存关系，箭头指向的结点依存于箭头发出的结点（父结点）。*两个结点没有连接关系表示两个随机变量能够在某些特定情况下条件独立，而两个结点有连接关系表示两个随机变量在任何条件下都不存在条件独立。*条件独立是贝叶斯网络所依赖的一个核心概念。每一个结点都与一个概率函数相关，概率函数的输入是该结点的父结点所表示的随机变量的一组特定值，输出为当前结点表示的随机变量的概率值。*概率函数值的大小表达的是结点之间依存关系的强度。*假设父结点有$n$个布尔变量，概率函数可表示成由$2^{n}$个条目构成的二维表，每个条目是其父结点各变量可能的取值（“T”或“F”）与当前结点真值的组合。

<img src="./img/page_106_1.jpg" width="600" />

构造贝叶斯网络涉及表示、推断和学习三个方面的问题

1. 表示：在某一随机变量的集合$x = \{X_{1}, \cdots, X_{n}\}$上给出其联合概率分布$P$。在贝叶斯网络表示中的主要问题是，即使在随机变量仅有两种取值，联合概率分布也需要对$\{X_{1}, \cdots, X_{n}\}$的所有$2^{n}$种不同取值下的概率情况进行说明，导致代价高昂。

2. 推断：由于贝叶斯网络是变量及其关系的完整模型，因此可以回答关于变量的询问，如观察到某些变量（证据变量）时，可以推断另一些变量子集的变化。在已知某些证据的情况下计算变量的后验分布的过程称作概率推理。

常用的精确推理方法包括**变量消除法**（variable elimination）和**团树**（clique tree）法。变量消除法[Zhang, Poole, 1996]的基本任务是计算条件概率$p(X_{Q} | X_{E} = x)$，其中，$X_{Q}$是询问变量的集合，$X_{E}$为已知证据的变量集合。其基本思想是通过分步计算不同变量的边缘分布按顺序逐个消除未观察到的非询问变量。团树法使用更全局化的数据结构调度各种操作，以获得更加有益的计算代价。

常用的近似推理算法有**重要性抽样法**（importance sampling）、**随机马尔可夫链蒙特卡罗**（Markov chain Monte Carlo，MCMC）模拟法、**循环信念传播法**（loopy belief propagation）和**泛化信念传播法**（generalized belief propagation）等。

3. 学习：参数学习的目的是决定变量之间相互关联的量化关系，即依存强度估计。即，对于每个结点$X$，计算给定父结点条件下$X$结点的概率，这些概率分布形式可以是任意的，通常是离散分布或高斯分布。

常用的参数学习方法包括**最大似然估计**法、**最大后验概率**法、**期望最大化**方法（EM）和**贝叶斯估计**方法。在贝叶斯图模型中使用较多的是贝叶斯估计法。

除了参数学习外，还有一项任务是寻找变量之间的图关系，即结构学习。在很简单的情况下贝叶斯网络可以由专家构造，但在多数实用系统中这一过程过于复杂，必须从大量数据中学习网络结构和局部分布的参数。自动学习贝叶斯网络的图结构是机器学习领域研究的一项颇具挑战性的任务。

贝叶斯网络是一种不定性因果关联模型，能够在已知有限的、不完整、不确定信息的条件下进行学习和推理，因此广泛应用于故障诊断和维修决策等领域。在自然语言处理中，贝叶斯网络已应用于汉语自动分词和词义消歧等任务。


## 6.3 马尔可夫模型

随机过程（随机函数）是随时间而随机变化的过程。

假设系统有$N$个有限状态$S = \{s_{1}, s_{2}, \cdots, s_{N}\}$，随时间的推移，该系统将从某一状态转移到另--状态。$Q = (q_{1}, q_{2}, \cdots, q_{T})$为一个随机变量序列，随机变量的取值为状态集$S$中的某个状态，假定在时间$t$的状态记为$q_{t}$。对该系统的描述通常需要给出当前时刻$t$的状态和其前面所有状态的关系：系统在时间$t$处于状态$s$的概率取决于其在时间$1, 2, \cdots, t - 1$的状态，该概率为

$$P(q_{t} = s_{j} | q_{t - 1} = s_{i}, q_{t - 2} = s_{k}, \cdots)$$

如果在特定条件下，系统在时间$t$的状态只与其在时间$t - 1$的状态相关，即

$$P(q_{t} = s_{j} | q_{t - 1} = s_{i}, q_{t - 2} = s_{k}, \cdots) = P(q_{t} = s_{j} | q_{t - 1} = s_{i}) \tag {6-1}$$

则该系统构成一个离散的一阶马尔可夫链（Markov chain）。

如果只考虑独立于时间$t$的随机过程，则：

$$P(q_{t} = s_{j} | q_{t - 1} = s_{i}) = a_{ij}, 1 \leq i, j \leq N \tag {6-2}$$

该随机过程为马尔可夫模型（Markov model）。其中，状态转移概率$a_{ij}$，必须满足以下条件：

$$a_{ij} \geq 0 \tag {6-3}$$

$$\sum_{j = 1}^{N} a_{ij} = 1 \tag {6-4}$$

$N$个状态的一阶马尔可夫过程有$N^{2}$个状态转移，可以表示成一个状态转移矩阵。

<img src="./img/page_109_1.jpg" width="600" />

对于$n$元语法模型，bi-gram为标准马尔可夫模型；当$n \geq 3$时，$n$元语法模型不符合马尔可夫模型的基本约束。不过由于$n$元语法模型的历史数量可以确定，通过将状态空间描述成多重前状态的交叉乘积的方式，可将其转换成马尔可夫模型，称其为$m$阶马尔可夫模型，$m$表示预测下一个状态所需的前状态个数，则$n$元语法模型为$n - 1$阶马尔可夫模型。


## 6.4 隐马尔可夫模型

在马尔可夫模型中，每个状态代表了一个可观察的事件，因此，马尔可夫模型又称作可视马尔可夫模型（visible Markov model，VMM）。隐马尔可夫模型（HMM）中，模型所经过的状态序列未知，只知道状态的概率函数，即观察到的事件是状态的随机函数，因此，该模型是一个双重随机过程。其中，模型的状态转换过程是不可观察的（隐蔽的），可观察事件的随机过程是隐蔽状态转换过程的随机函数。

<img src="./img/fig_6_5.jpg" width="400" />

<img src="./img/page_110_1.jpg" width="600" />

HMM模型可表示为一个五元组$\mu = (S, K, \mathbf{A}, \mathbf{B}, \mathbf{\pi})$，其中$S$为状态的集合，$K$为输出符号的集合，$\mathbf{\pi}$、$\mathbf{A}$和$\mathbf{B}$分别表示初始状态的概率分布、状态转移概率和符号发射概率。有时也将其简记为三元组$\mu = (\mathbf{A}, \mathbf{B}, \mathbf{\pi})$。

1. 模型中的状态数$N = |S|$；

2. 每个状态可能输出的符号数$M = |K|$；

3. 状态转移概率矩阵$\mathbf{A} = \{ a_{ij} \}$，其中

$$\begin{aligned}
& a_{ij} = P(q_{t} = s_{j} | q_{t - 1} = s_{i}) \\
& a_{ij} \geq 0 \\
& \sum_{j = 1}^{N} a_{ij} = 1 \\
& 1 \leq i, j \leq N \\
\end{aligned} \tag {6-5}$$

4. 从状态$s_{j}$观察到符号$v_{k}$的概率分布矩阵$\mathbf{A} = \{ b_{j}(k) \}$，其中

$$\begin{aligned}
& b_{j}(k) = P(O_{t} = v_{k} | q_{t} = s_{j}) \\
& b_{j}(k) \geq 0 \\
& \sum_{k = 1}^{M} b_{j}(k) = 1 \\
& 1 \leq j \leq N, \ 1 \leq k \leq M \\
\end{aligned} \tag {6-6}$$

观察符号的概率又称符号发射概率（symbol emission probability）；

5. 初始状态概率分布$\mathbf{\pi} = \{ \pi_{i} \}$，其中，

$$\begin{aligned}
& \pi_{i} = P(q_{1} = s_{i}) \\
& \pi_{i} \geq 0 \\
& \sum_{i = 1}^{N} \pi_{i} = 1 \\
& 1 \leq i \leq N \\
\end{aligned} \tag {6-7}$$

考虑潜在事件随机地生成表面事件，假设给定HMM模型$\mu = (\mathbf{A}, \mathbf{B}, \mathbf{\pi})$，则观察序列$O = O_{1} O_{2} \cdots O_{T}$可以由以下步骤产生：

1. 根据初始状态的概率分布$\mathbf{\pi}$，选择一个初始状态$q_{1} = s_{i}$；

2. 设$t = 1$；

3. 根据状态$s_{i}$的输出概率分布$b_{i}(k)$输出$O_{t} = v_{k}$；

4. 根据状态转移概率分布$a_{ij}$，将当前时刻$t$的状态转移到新的状态$q_{t + 1} = s_{j}$；

5. $t = t + 1$，如果$t \lt T$，重复执行步骤（3）、（4），否则，结束算法。

HMM的三个基本问题：

1. 估计问题：给定观察序列$O = O_{1} O_{2} \cdots O_{T}$和模型$\mu = (\mathbf{A}, \mathbf{B}, \mathbf{\pi})$，计算观察序列$O$的概率，即$P( O | \mu)$；

2. 序列问题：给定观察序列$O = O_{1} O_{2} \cdots O_{T}$和模型$\mu = (\mathbf{A}, \mathbf{B}, \mathbf{\pi})$，选择在一定意义下“最优”的状态序列$Q = q_{1} q_{2} \cdots q_{T}$，使得该状态序列“最好地解释”观察序列；

3. 训练问题（参数估计问题）：给定一个观察序列$O = O_{1} O_{2} \cdots O_{T}$，根据最大似然估计计算模型参数值，即计算使$P(O | \mu)$最大时，模型$\mu = (\mathbf{A}, \mathbf{B}, \mathbf{\pi})$的参数。

三个基本问题可通过前后向算法及参数估计解决。

### 6.4.1 求解观察序列的概率

**解码（decoding）**问题：给定观察序列$O = O_{1} O_{2} \cdots O_{T}$和模型$\mu = (\mathbf{A}, \mathbf{B}, \mathbf{\pi})$，计算观察序列$O$的概率，即$P(O | \mu)$。

穷举法：对于任意的状态序列$Q = q_{1} q_{2} \cdots q_{T}$，有

$$\begin{aligned}
P(O| Q ; \mu) & = \prod_{t = 1}^{T - 1} P(O_{t} | q_{t}, q_{t - 1}; \mu) \\
& = b_{q_{1}}(O_{1}) b_{q_{2}}(O_{2}) \cdots b_{q_{T}}(O_{T})
\end{aligned} \tag {6-8}$$

$$\begin{aligned}
P(Q ; \mu) & = \pi_{q_{1}} a_{q_{1} q_{2}} a_{q_{2} q_{3}} \cdots a_{q_{T - 1} q_{T}}
\end{aligned} \tag {6-9}$$

$$\begin{aligned}
P(O ; \mu) & = \sum_{Q} P(O, Q; \mu) \\
& = \sum_{Q} P(O | Q; \mu) P(Q ; \mu) \\
& = \sum_{Q} \left( \pi_{q_{1}} b_{q_{1}}(O_{1}) \prod_{t = 1}^{T - 1} a_{q_{t} q_{t + 1}} b_{q_{t + 1}}(O_{t + 1}) \right)
\end{aligned} \tag {6-11}$$

该方式的问题是计算时间复杂度呈指数增长，假高模型$\mu = (\mathbf{A}, \mathbf{B}, \mathbf{\pi})$有$N$个不同的状态，时间长度为$T$，则可能状态序列数量为$N^{T}$。前向算法（前向计算过程，forward procedure）通过动态规划方式使“指数爆炸”问题可以在时间复杂度为$\mathcal{O}(N^{2} T)$的范围内解决。HMM的动态规划问题--般用格架[Manning、Schitze，1999]（trellis，lattice）组织形式描述。对于一个在某一时间结束在一定状态的HMM，每一个格（节点）记录该HMM所有输出符号的概率，较长子路径的概率可以由较短子路径的概率计算出来。

<img src="./img/fig_6_6.jpg" width="400" />

**定义6-1（前向变量$\alpha_{t}(i)$）**在$t$时刻，HMM输出序列为$O = O_{1} O_{2} \cdots O_{T}$，并且位于状态$s_{i}$的概率：

$$\alpha_{t}(i) = P(O_{1} O_{2} \cdots O_{t}, q_{t} = s_{t}; \mu) \tag {6-12}$$

由于&P(O ; \mu)&是在所有状态$q_{T}$下观察到序列$O = O_{1} O_{2} \cdots O_{T}$的概率：

$$P(O ; \mu) = \sum_{s_{i}} P(O_{1} O_{2} \cdots O_{T}, q_{t} = s_{t}; \mu)$$

在前向算法中,采用动态规划的方法计算前向变量a,(i),其实现思想基于如下观察:
在时间t+1的前向变量可以根据在时间t时的前向变量a,(1),a,(2),..,a,(N)的值来归
纳计算:
ar+t(j)= ( Za(i)a, )b,(O+)
(6-14)
在格架结构中,a+1(j)存放在(s;,t+1)处的结点上,表示在已知观察序列0.O2...
0,0.+1的情况下，从时间l到达下一一个时间t+1时状态为s;的概率。图6-7描述了
式(6-14)的归纳关系。
从初始时间开始到l+ 1,HMM到达状态s;,并
a,(1)
Po,
90. 
输出观察序列020...0.+1的过程可以分解为以下a,2)
两个步骤:
83. o
O's,
a,(3)
(1)从初始时间开始到时间t,HMM到达状态
s;,并输出观察序列0O2...0, ; .
(2)从状态s,转移到状态s;,并在状态s,输a,(N) 
1+1
出Or+1。
a(i)
(r+()
这里s;可以是HMM的任意状态。根据前向
图6-7前向变量的归纳关系
变量a,(i)的定义,从某一个状态s,出发完成第一步的概率就是a,(i),而实现第二步的
概率为aj Xb;(O+1)。因此，从初始时间到t+1整个过程的概率为: a,(i)Xa; X
b;(O+1)。由于HMM可以从不同的s;转移到s;,一共有N个不同的状态,因此,得到了
式(6-14)。
根据式(6-14)给出的归纳关系,可以按时间顺序和状态顺序依次计算前向变量
a:(x),a2(x),..,ar(x)(x为HMM的状态变量)。由此,得到如下前向算法。
